In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import networkx as nx

import re

In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2013, 2016]

# Ignore words that appear at a frequency more than max_frequ in the corpus
max_frequ = 0.8

# Ignore words appearing less than min_appear in the whole corpus
min_appear = 20

# Range fo cluster number you want to test
cluster_ranges = range(2, 3) #range(2, 100) # Warning, long to compute (but nice)

# Number of trial you want to do for each test
nb_trial_by_test = 3

# Number of cluster you finally choose
nb_cluster = 5

# Max iteration for each kmeans (default: 300)
max_iter = 300

# Number of labels by cluster
nb_labels = 7

In [4]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw
sourceFST += [x.replace('\n', '') for x in open('perso_words-fr.txt', mode="r", encoding="utf-8").readlines()]

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    arti_lower = article.lower()
    arti_2words = re.sub(" [0-z][0-z] ", " ", arti_lower) # word of length < 2
    arti_e = re.sub("(é|è|ê)", "e", arti_2words)
    arti_o = re.sub("à", "a", arti_e)
    arti_i = re.sub("ô", "o", arti_o)
    artiregex = re.sub("î", "i", arti_i)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [ ]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [ ]:
# Let's process our corpus, and determine a limit to split it in partitions

# limits[] let us know when to delimit partitions
temp_usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if isinstance(data['abstract'][i], float) or isEnglish(data['abstract'][i]):
        temp_usable.append([])
    else:
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        temp_usable.append(re.sub(" [0-z][0-z] ", " ", stemmer.stem(lemmatize(text))))



In [ ]:
# Post-process word removal
post_words = [x.replace('\n', '') for x in open('post_process_words-fr.txt', mode="r", encoding="utf-8").readlines()]

for i in range(0, len(temp_usable)):
    if len(temp_usable[i]) != 0:
        arti = temp_usable[i].split()
        res = []
        for word in arti:
            if word not in post_words:
                res.append(word)
        temp_usable[i] = ' '.join(res)

In [ ]:
# Limit computing

# limits[] let us know when to delimit partitions
# the number into limits represent the document numero that is not included in the previous bound
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(temp_usable)):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(temp_usable[i], float) and not len(temp_usable[i]) == 0 and not isEnglish(temp_usable[i]):
        usable.append(temp_usable[i])
        numArti += 1
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
limits.append(numArti)

In [ ]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=max_frequ, min_df=min_appear)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

usable

In [ ]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

# KMeans & Silhouette Score

In [ ]:
# Applying KMeans on tfidf
# the labels_ give assignment of doc to the cluster number 


In [ ]:
# SOURCE :
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html


# Silhouette analysis can be used to study the separation distance between the resulting clusters.
# The silhouette plot displays a measure of how close each point in one cluster is to points in 
# the neighboring clusters and thus provides a way to assess parameters like number of clusters visually.
# This measure has a range of [-1, 1].

from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [ ]:
def kmeans_silhouette(samples, k):
    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(random_state = 1 ,n_clusters=k, max_iter=800, init='k-means++', n_init=50, n_jobs=-1)
    cluster_labels = clusterer.fit_predict(samples)

    silhouette_avg = silhouette_score(samples, cluster_labels)
    sample_silhouette_values = silhouette_samples(samples, cluster_labels)

    return {'k':k, 'cluster':clusterer, 'labels':cluster_labels ,'silhouette_avg':silhouette_avg, 'sample_silhouette_values':sample_silhouette_values }


def kmeans_silhouette_range(samples, k_min, k_max):
    kmeans_silhouette_range_value = {}
    for k in range(k_min, k_max):
        kmeans_silhouette_range_value[k] = kmeans_silhouette(samples, k)
    return kmeans_silhouette_range_value


In [ ]:
def display_kmeans_silhouette(kmeans_silhouette_value):
    clusterer = kmeans_silhouette_value['cluster']
    cluster_labels = kmeans_silhouette_value['labels']
    silhouette_avg = kmeans_silhouette_value['silhouette_avg']
    sample_silhouette_values = kmeans_silhouette_value['sample_silhouette_values']
    n_cluster = kmeans_silhouette_value['k']

    print("For n_cluster =", n_cluster,
              "The average silhouette_score is :", silhouette_avg)

    # Create a subplot with 1 row and 2 columns
    fig, (ax1) = plt.subplots(1, 1)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(cluster_labels) + (n_cluster + 1) * 10])

    y_lower = 10
    for i in range(n_cluster):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_cluster)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    plt.suptitle(("Silhouette analysis for cluster = %d" % n_cluster),
                 fontsize=14, fontweight='bold')
    plt.show()

In [ ]:
X = tfidf.toarray()
k_s_r = kmeans_silhouette_range(X, 4, 12)

In [ ]:




k_s= kmeans_silhouette(X, 10)

In [ ]:
display_kmeans_silhouette(k_s)

In [ ]:
from matplotlib.ticker import FuncFormatter
import pandas as pd
import numpy as np


def display_time_distribution_cluster(labels, labels_string):
    label_limit = ['2004-2007', '2007-2010','2010-2013','2013-2016', '2016-2018']
    limit_index = 0

    zeros = np.zeros((10, 5))
    df = pd.DataFrame(zeros, columns=label_limit)

    for i in range(len(labels)):
        if i <= limits[limit_index]:
            df.at[labels[i],label_limit[limit_index]] += 1
        else:
            limit_index += 1
            df.at[labels[i],label_limit[limit_index]] += 1

    # To percent
    df = df.apply(lambda x: x / x.sum(), axis=1)

    # Plot
    df.plot.barh(figsize=(20, 30), width=0.9)
    plt.title('Time distribution by cluster', fontsize= 24, color='gray')

    ## PlotSwagg ##
    plt.yticks(fontsize=14, rotation=0, color='gray')
    plt.xticks(fontsize=14, rotation=0, color='gray')

    # Cleanest Percent
    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:.0%}'.format(x))) 
    plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{}'.format(" ".join(labels_string[y]))))

    # Less border
    plt.gca().xaxis.grid(True)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.gca().spines['left'].set_edgecolor('gray')
    plt.gca().spines['right'].set_edgecolor('gray')

    # Percent line under the barH
    plt.gca().set_axisbelow(True)

    plt.show()


In [ ]:

# it looks like -> { doc_number : [partition_number, cluster_number] }
doc_clustering = {}

cluster_labels = k_s['labels']

numDoc = 0
for i in range(0, len(limits)):

    previousBound = 0
    if i > 0:
        previousBound = limits[i-1]
    clusterer = kmeans_silhouette(partitions_tfidf[i], nb_cluster)["labels"]
    for numDocItern in range(0, limits[i]-previousBound):
        doc_clustering[numDoc] = [i, clusterer[numDocItern]]
        numDoc+=1



In [ ]:
len(doc_clustering)

In [ ]:
# Allows to get list of documents number
# return [dou numbers]
# params : partition_number , cluster number
def get_doc(part, clust):
    docs = []
    for i in range(0,len(doc_clustering)):
        if doc_clustering[i][0] == part and doc_clustering[i][1] == clust:
            docs.append(i)
    return docs

In [ ]:
# Get the partitions variable
# Here partitions[part][cluster] = list of docs numbe
partitions = []
for i in range(0, len(limits)):
    clusters = []
    for j in range(0, nb_cluster):
        clusters.append(get_doc(i,j))
    partitions.append(clusters)

In [ ]:
partitions

# Khi²

In [ ]:
# tf_of_your_word = tf[numDoc][strWord]
tf = []
for doc in usable:
    tf_doc = {}
    for word in vectorizer.get_feature_names():
        tf_doc[word] = doc.count(word)
    tf.append(tf_doc)

In [ ]:
# Number total of words
# nb_total_word[numPartition]
nb_total_word = []
nb = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        nb += tf[numDoc][word]
    if numDoc+1 in limits:
        nb_total_word.append(nb)
        nb=0
    

In [ ]:
nb_total_word

In [ ]:
tf[0]

In [ ]:
# nb_word[num_partition][word]
nb_word = []

word_in_this_parti = {}
for word in vectorizer.get_feature_names():
    word_in_this_parti[word] = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        word_in_this_parti[word] += tf[numDoc][word]
    if numDoc+1 in limits:
        nb_word.append(word_in_this_parti)
        word_in_this_parti = {}
        for word in vectorizer.get_feature_names():
            word_in_this_parti[word] = 0

In [ ]:
len(nb_word)

In [ ]:
# nb_word_by_cluster[numPartition][numCluster]
nb_word_by_cluster = []
for parti in partitions:
    nb_word_clus = []
    for cluster in parti:
        nb = 0
        for numDoc in cluster:
            for word in vectorizer.get_feature_names():
                nb += tf[numDoc][word]
        nb_word_clus.append(nb)
    nb_word_by_cluster.append(nb_word_clus)

In [ ]:
# value_of_khi2 = khi2[numPartition][numCluster][word]
khi2 = []

for numParti in range(0, len(partitions)):
    khi2parti = []
    for numCluster in range(0, len(partitions[numParti])):
        khi2cluster = {}
        
        for word in vectorizer.get_feature_names():
            if nb_word_by_cluster[numParti][numCluster] == 0:
                khi2cluster[word] = 0
            else:
                word_in_this_parti[word] = 0
                E = nb_word[numParti][word]
                E =+ nb_word_by_cluster[numParti][numCluster]
                E = E/ nb_total_word[numParti]
                N = 0
                for numDoc in partitions[numParti][numCluster]:
                    N += tf[numDoc][word]
                khi2cluster[word] = (pow(N - E, 2)/E)        
        khi2parti.append(khi2cluster)
    khi2.append(khi2parti)

In [ ]:
# list of your labels = labels[numPartition][numCluster]
labels = []

for numPartition in range(0, len(nb_word_by_cluster)):
    label_clus = []
    for numCluster in range(0, len(nb_word_by_cluster[numPartition])):
        label_clus.append(Counter(khi2[numPartition][numCluster]).most_common(nb_labels))
    labels.append(label_clus)

In [ ]:
# Some clusters can be empty, so they have a score of 0 on each labels
labels

# Diachronic Analysis

In [ ]:
labels[0]

In [ ]:
labels_cluster = []

for i in range(0, len(limits)):
    labels_cluster.append('')
    s = []
    for j in range(0, nb_cluster):
        s.append(' '.join([t[0] for t in labels[i][j]]))
    labels_cluster.append(s)
labels_cluster

In [ ]:
# Low level analysis

display_time_distribution_cluster(k_s['labels'], labels_cluster)

In [ ]:

def inter(listA, listB):
    return np.intersect1d(listA, listB)
    
# cluster_t and cluster_s must be in two different partitions
def proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S):
    total_inter = 0
    total_t = 0
    for f in range(0, len(labels[num_partition_T][num_cluster_t])):
        for f_s in labels[num_partition_S][num_cluster_s]:
            if labels[num_partition_T][num_cluster_t][f][0] == f_s[0]:
                total_inter += labels[num_partition_T][num_cluster_t][f][1]
                break
        total_t += labels[num_partition_T][num_cluster_t][f][1]
    if total_t == 0:
        return 0
    return total_inter / total_t
    

def P_A(num_cluster_s, num_partition_T, num_partition_S):
    # first, we have to know what are the cluster which got the label
    total = 0
    nb_computation = 0
    for label_s in labels[num_partition_S][num_cluster_s]:
        for num_cluster_t in range(0, len(partitions[num_partition_T])):
            if label_s in labels[num_partition_T][num_cluster_t]:
                total += proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S)
                nb_computation += 1
    if nb_computation == 0:
        return 0
    return total / nb_computation

# Define a coeficient for the activity 
def activity(num_partition_S, num_partition_T):
    res = 0
    for num_cluster_s in range(0, len(partitions[num_partition_S])):
        res += P_A(num_cluster_s, num_partition_T, num_partition_S)
    return res / len(partitions[num_partition_S])

# Ecart-type, but it isn't very usefull xD
sigma_t = 0.01
sigma_s = 0.01

# Our Graal
def similar(num_cluster_t, num_partition_T, num_cluster_s, num_partition_S):
    cond1 = proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S) > P_A(num_cluster_s, num_partition_T, num_partition_S)
    cond2 = proba(num_cluster_t, num_cluster_s, num_partition_T, num_partition_S) > activity(num_partition_S, num_partition_T) + sigma_s
    
    cond3 = proba(num_cluster_s, num_cluster_t, num_partition_S, num_partition_T) > P_A(num_cluster_t, num_partition_S, num_partition_T)
    cond4 = proba(num_cluster_s, num_cluster_t, num_partition_S, num_partition_T) > activity(num_partition_T, num_partition_S) + sigma_t
    return cond1 and cond2 and cond3 and cond4
    

In [ ]:
# Plot the diachronic analysis results

# Node coloring source: https://stackoverflow.com/questions/13517614/draw-different-color-for-nodes-in-networkx-based-on-their-node-value#13517947

# The more the partition is high, the more the partition is recent

val_map = {}
values = []

g = nx.Graph() #nx.DiGraph(directed=True)
for n_part in range(0, len(limits)):
    for n_clus in range(0, nb_cluster):
        node_str = '('+str(n_clus)+','+str(n_part)+')'
        g.add_node(node_str, posxy=(n_part, n_clus), partition = n_part)
        val_map[node_str] = n_part/len(limits)

values = [val_map.get(node, 0.25) for node in g.nodes()]

for numParti in range(0, len(partitions)-1):
    for num_cluster_t in range(0, nb_cluster):
        for num_cluster_s in range(0, nb_cluster):
            if not similar(num_cluster_t, numParti, num_cluster_s, numParti+1):
                #print("("+str(num_cluster_t)+","+str(numParti)+") est similaire à ("+str(num_cluster_s)+","+str(numParti+1)+")")
                g.add_edges_from([("("+str(num_cluster_t)+","+str(numParti)+")", "("+str(num_cluster_s)+","+str(numParti+1)+")")])
                
positions = nx.get_node_attributes(g,'posxy')
options = {
    'node_color': 'blue',
    'node_size': 100,
    'width': 3,
    'arrowstyle': '-|>',
    'arrowsize': 12,
}
nx.draw(g, positions, node_size=3000/nb_cluster, cmap=plt.get_cmap('jet'), node_color=values, arrowsize=20, arrows=True, arrowstyle="-|>", arraowsize=12)
pos = nx.circular_layout(g)


In [ ]:
for labels_clus in labels:
    for clus in labels_clus:
        for w in clus:
            print(w[0])
        print('')
    print('___')